# 01_import dataset

## 01-1_import data

In [1]:
import os
import json
import pandas as pd

#https://drive.google.com/drive/folders/18qV82fNY3IIWu3BRoGqm_LNgJzE8Akbr?usp=drive_link
#base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"
base_dir= '/Users/andyw.p.chen/Documents/Project/datasets'
#base_dir=  "c:\Users\user\Downloads\datasets"

def load_json_to_df(filename: str) -> pd.DataFrame:
    file_path = os.path.join(base_dir, filename)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 如果是 { "target": {id: value, ...} }
    if isinstance(data, dict) and len(data) == 1 and isinstance(next(iter(data.values())), dict):
        key, inner = next(iter(data.items()))
        return pd.DataFrame(list(inner.items()), columns=["id", key])

    # dict of scalar
    if isinstance(data, dict):
        return pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])

    # list of dict
    elif isinstance(data, list):
        return pd.DataFrame(data)

    else:
        raise ValueError(f"Unsupported JSON structure in {filename}: {type(data)}")


def load_csv_to_df(filename: str) -> pd.DataFrame:
    """讀取 CSV 並轉為 DataFrame。"""
    return pd.read_csv(os.path.join(base_dir, filename))

# JSON 資料
##mcc_codes_df = load_json_to_df("mcc_codes.json")
train_fraud_labels_df = load_json_to_df("train_fraud_labels.json")

# CSV 資料
cards_df = load_csv_to_df("cards_data.csv")
transactions_df = load_csv_to_df("transactions_data.csv")
users_df = load_csv_to_df("users_data.csv")

# 簡單檢查
#print(mcc_codes_df.head())
#print(train_fraud_labels_df.head())
#print(cards_df.head())
#print(transactions_df.head())
#print(users_df.apthead())


## 01-2_rename variable in each data set

In [2]:
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'id': 'transactions_id'})
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'target': 'is_fraud'})

cards_df = cards_df.rename(columns={'id':'card_id'})

users_df = users_df.rename(columns={'id':'client_id'})

transactions_df = transactions_df.rename(columns={'mcc': 'mcc_code'})
transactions_df = transactions_df.rename(columns={'id': 'transaction_id'})




## 01-3_變數型態統一及缺失值處理

In [3]:
def add_missing_flags(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    在 DataFrame 中對指定欄位建立 missing flag 欄位
    flag=1 表示缺失值，flag=0 表示非缺失值
    
    參數
    ----
    df : pd.DataFrame
        輸入的資料框
    cols : list
        要檢查的欄位名稱清單
    
    回傳
    ----
    pd.DataFrame : 新的資料框 (含新增的 flag 欄位)
    """
    for col in cols:
        df[f"{col}_missing_flag"] = df[col].isna().astype(int)
    return df

transactions_df = add_missing_flags(transactions_df, ["merchant_state", "zip", "errors"])

In [4]:
##train_fraud_labels_df##
train_fraud_labels_df["is_fraud"]=train_fraud_labels_df["is_fraud"].astype("category") 
train_fraud_labels_df["transactions_id"]=train_fraud_labels_df["transactions_id"].astype(int) #合併資料需要

##cards_df##
cards_df["card_brand"]=cards_df["card_brand"].astype("category") 
cards_df["card_type"]=cards_df["card_type"].astype("category")
#####不要load這行 cards_df["expires"]=pd.to_datetime(cards_df["expires"], format="%m/%Y")
cards_df["expires"] = pd.to_datetime(cards_df["expires"], format="%m/%Y").dt.to_period("M")
cards_df["has_chip"]=cards_df["has_chip"].astype("category")

cards_df['credit_limit'] = cards_df['credit_limit'].replace(r'[\$,]', '', regex=True).astype(int)
#####不要load這行 cards_df["acct_open_date"]=pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y")
cards_df["acct_open_date"] = pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y").dt.to_period("M")
#####不要load這行 cards_df["year_pin_last_changed"]=pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y")
cards_df["year_pin_last_changed"] = pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y").dt.to_period("Y")
cards_df["card_on_dark_web"]=cards_df["card_on_dark_web"].astype("category") 

##users_df##
users_df["birth_year"] = pd.to_datetime(users_df["birth_year"], format="%Y").dt.to_period("Y")
users_df["birth_month"] = pd.to_datetime(users_df["birth_month"], format="%m").dt.to_period("M")
users_df["gender"]=users_df["gender"].astype("category") 
users_df['per_capita_income'] = users_df['per_capita_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['yearly_income'] = users_df['yearly_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['total_debt'] = users_df['total_debt'].replace(r'[\$,]', '', regex=True).astype(int)

##transactions_df##
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
#浮點數轉整數原因確定？
transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float).astype(int)
##負數取log調成1
#transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float)

transactions_df["use_chip"]=transactions_df["use_chip"].astype("category") 

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'merchant_state'
] = 'online'

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'zip'
] = 20000 #原本是-1
## 我沒有全部改，這樣完之後仍有89006筆Missing，剩下都是在國外
transactions_df['zip'] = transactions_df['zip'].fillna(10000) #原本是-999
transactions_df["zip"]=transactions_df["zip"].astype("int64")

transactions_df['errors'] = transactions_df['errors'].astype('category')
transactions_df['errors'] = transactions_df['errors'].cat.add_categories('No_error').fillna('No_error')



In [5]:
#cars one hot encoding
##統一類別變數轉dummy variable(要注意共線性問題，應刪掉其中之一)

#card_type 原始種類：Debit_57%, Credit_33%, Debit(Prepaid)_9%
#card_brand 原始種類：MasterCard_52%, Visa_38%, Amex_7%, Discovery_3%
#has_chip 原始種類：Yes_89%, No_11%
#card_on_dark_web 原始種類：No_0%
cols_to_encode = ['card_type', 'card_brand', 'has_chip']
cards_df[cols_to_encode] = cards_df[cols_to_encode].astype('category')
dummies_cards = pd.get_dummies(
    cards_df[cols_to_encode], 
    prefix=cols_to_encode, 
    dtype='uint8'
    )
cards_df = pd.concat([cards_df, dummies_cards], axis=1)

#use_chip 原始種類：Swiped_52%, Chipe_36%, Online_12%
dummies_use = pd.get_dummies(transactions_df['use_chip'], prefix='use_chip', dtype='uint8')
transactions_df = pd.concat([transactions_df, dummies_use], axis=1)

#gender 原始種類：Female_51%, Male_49%
dummies_gender = pd.get_dummies(users_df['gender'], prefix='gender', dtype='uint8')
users_df = pd.concat([users_df, dummies_gender], axis=1)


cards_df.drop(columns=["has_chip_NO","has_chip"], inplace=True)
transactions_df.drop(columns=["use_chip"], inplace=True)
users_df.drop(columns=["gender_Female"], inplace=True)

# 02_資料整併成一張dataframe

## 02-1_資料整併

In [8]:
#transactions_df.loc[transactions_df["transaction_id"] == 10649266] #transaction_id vs id

#原始資料筆數：13305915
### transactions_df+train_fraud_labels_df      left 會有4390952 missing values
merged = pd.merge(transactions_df, train_fraud_labels_df, left_on="transaction_id", right_on="transactions_id", how="outer")
### transactions_df train_fraud_labels_df(8914963) + users_df 對過去不會有missing values
merged = pd.merge(merged,users_df , left_on="client_id", right_on="client_id", how="left")
### transactions_df train_fraud_labels_df users_df + cards_df 對過去不會有missing values
merged = pd.merge(merged,cards_df , left_on="card_id", right_on="card_id", how="left")

#刪掉重複的columns
merged.drop(columns=["transactions_id"], inplace=True)
merged.drop(columns=["client_id_y"], inplace=True)

## 合併完之後最後處理is_fraud(原會有missing values問題)
merged["is_fraud"] = merged["is_fraud"].astype(str)
merged.loc[merged['is_fraud'].str.lower() == 'no','is_fraud'] = '0'
merged.loc[merged['is_fraud'].str.lower() == 'yes','is_fraud'] = '1'
merged["is_fraud"] = pd.to_numeric(merged["is_fraud"], errors="coerce").astype("Int64")

merged = add_missing_flags(merged, ["is_fraud"])

#merged.to_csv("merged.csv", index=False)

# 先刪除不需要的DataFrame以節省記憶體
del transactions_df, users_df, cards_df, train_fraud_labels_df, cols_to_encode, dummies_cards, dummies_use, dummies_gender

NameError: name 'transactions_df' is not defined

In [ ]:
backup_merged = merged.copy()
#merged = backup_merged.copy()

# 04_RFM features engineering model

## 04-1_資料進行變數轉換以求模型配飾更佳表現

In [ ]:
##有出事再趕快回復原狀
merged = backup_merged.copy()

In [9]:
import numpy as np
import pandas as pd

# 確保日期是 datetime 並排序
merged['date'] = pd.to_datetime(merged['date'])
merged = merged.sort_values(by=['client_id_x', 'date']).reset_index(drop=True)

# --- RecencyInterval ---
merged['RecencyInterval'] = merged.groupby('client_id_x')['date'].diff().dt.total_seconds().fillna(0)/60

# --- TxnFrequency for multiple windows (向量化滑動窗口) ---
window_days = [7, 30, 60, 90]
for w in window_days:
    merged[f'TxnFrequency_{w}d'] = 0

def compute_freq_vectorized(dates, windows):
    """向量化計算每筆交易在每個 window 內的交易數"""
    n = len(dates)
    dates_int = dates.values.astype('datetime64[D]').astype(int)
    res = {w: np.zeros(n, dtype=int) for w in windows}
    for w in windows:
        left = 0
        counts = np.zeros(n, dtype=int)
        for right in range(n):
            while dates_int[right] - dates_int[left] > w:
                left += 1
            counts[right] = right - left + 1
        res[w] = counts
    return res

# 分組計算
for cid, g in merged.groupby('client_id_x', sort=False):
    freq_dict = compute_freq_vectorized(g['date'], window_days)
    for w in window_days:
        merged.loc[g.index, f'TxnFrequency_{w}d'] = freq_dict[w]

# --- AmtDelta ---
merged['prev_amount'] = merged.groupby('client_id_x')['amount'].shift(1)
merged['AmtDelta'] = merged['amount'] - merged['prev_amount']
merged['AmtDelta'] = merged['AmtDelta'].fillna(0)
merged.drop(columns='prev_amount', inplace=True)

In [10]:
# US region mapping
us_region_map = {
    'Northeast': ['NY','NJ','PA','MA','CT','RI','NH','VT','ME'],
    'Midwest': ['IL','OH','MI','IN','WI','MN','IA','MO','ND','SD','NE','KS'],
    'South': ['FL','GA','SC','NC','AL','MS','LA','TX','OK','TN','KY','VA','WV','AR','MD','DE','DC'],
    'West': ['CA','WA','OR','NV','AZ','NM','CO','UT','ID','MT','WY','AK','HI'],
}
continent_map = {
    'Europe': [ ... ],  # 原本 continent_map['Europe'] 可直接使用
    'Online': ['online','AA']
}

us_region_lookup = {state: region for region, states in us_region_map.items() for state in states}

# --- 向量化 location 特徵 ---
merged['merchant_online'] = merged['merchant_state'].eq('online').astype('uint8')
merged['merchant_us'] = merged['merchant_state'].isin(us_region_lookup.keys()).astype('uint8')
merged['merchant_eu'] = merged['merchant_state'].isin(continent_map['Europe']).astype('uint8')
merged['merchant_others'] = (~merged[['merchant_online','merchant_us','merchant_eu']].any(axis=1)).astype('uint8')


In [11]:
# --- 首次交易標記 ---
merged['FirstTxnInRegion'] = (~merged.duplicated(subset=['client_id_x', 'merchant_state'])).astype('uint8')

In [12]:
# DifferentState
merged['prev_state']=(merged
                     .groupby('client_id_x')['merchant_state']
                     .shift(1))

merged['DifferentState'] = (
    (merged['merchant_state'] != merged['prev_state'])
    & merged['prev_state'].notna()
).astype(int)

merged = merged.drop(columns=['prev_state'])


In [13]:
#merged[["card_id","card_number"]]
import numpy as np
from scipy import stats 

# === (1) log轉換 ===
merged['amount'] = np.where(merged['amount'] < 0, 0, merged['amount'])  # 負數變 0
merged['amount'] = np.log(merged['amount'] + 1)  

# === (3) 平方根轉換 ===
merged['credit_limit']=np.sqrt(merged['credit_limit'])
merged['total_debt']=np.sqrt(merged['total_debt'])

# === (3) 立方根轉換 ===
merged['yearly_income']=np.cbrt(merged['yearly_income'])
merged['per_capita_income']=np.cbrt(merged['per_capita_income'])

## Box-Cox Transformation
###merged['yearly_income'], fitted_lambda = stats.boxcox(merged['yearly_income'])

# === (5) Yeo–Johnson 轉換（可處理負值） ===
###merged['per_capita_income'], lambdaValue =stats.yeojohnson(merged['per_capita_income'])

## 04-2_分割訓練集及測試集

In [14]:
# --- 選取數值型變數 ---
num_cols = merged.select_dtypes(include=['int64', 'float64','uint8','datetime64[ns]']).columns
df2 = merged[num_cols]

# --- dropna ---
df_cleaned = df2.dropna()
del df2

# --- 避免共線性 ---
df_cleaned.drop(columns=["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction"], inplace=True)

# --- 確保 date 欄位在 df_cleaned 中 ---
if 'date' not in df_cleaned.columns:
    df_cleaned['date'] = merged.loc[df_cleaned.index, 'date']

# --- 依時間排序 ---
df_sorted = df_cleaned.sort_values('date')
df_sorted['year'] = df_sorted['date'].dt.year

# 2010–2011 → block 0
# 2012–2013 → block 1
# ...
# 2018–2019 → block 4（如果你真的是 2010–2019 共 10 年，會有 5 個 block）
df_sorted['time_block'] = (df_sorted['year'] - 2010) // 1


#對每個時間block做切割
train_list = []
test_list = []

for block_id, block_df in df_sorted.groupby('time_block'):
    print(f"\nProcessing Block {block_id}")

    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)

    train_block = block_df.iloc[:split_index].copy()
    test_block  = block_df.iloc[split_index:].copy()

    # -----------------------------
    # 1️⃣ 用「該 block 的 train」算 fraud rate
    # -----------------------------
    fraud_rate = (
        train_block
        .groupby('mcc_code')['is_fraud']
        .mean()
    )

    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index

    # -----------------------------
    # 2️⃣ 套用到該 block 的 train / test
    # -----------------------------
    train_block['HighRiskMCC'] = train_block['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_block['HighRiskMCC']  = test_block['mcc_code'].isin(high_risk_mcc).astype('uint8')

    train_list.append(train_block)
    test_list.append(test_block)


train_df = pd.concat(train_list).drop(columns=['date', 'year', 'time_block'])
test_df  = pd.concat(test_list).drop(columns=['date', 'year', 'time_block'])


#比例檢查
for block_id in sorted(df_sorted['time_block'].unique()):
    print(f"\nBlock {block_id}")
    print("Train fraud count:")
    print(train_df.loc[df_sorted['time_block'] == block_id, 'is_fraud'].value_counts())
    print("Test fraud count:")
    print(test_df.loc[df_sorted['time_block'] == block_id, 'is_fraud'].value_counts())





Processing Block 0

Processing Block 1

Processing Block 2

Processing Block 3

Processing Block 4

Processing Block 5

Processing Block 6

Processing Block 7

Processing Block 8

Processing Block 9

Block 0
Train fraud count:
is_fraud
0    663110
1      2113
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    165846
1       460
Name: count, dtype: Int64

Block 1
Train fraud count:
is_fraud
0    690705
1        37
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    172686
Name: count, dtype: Int64

Block 2
Train fraud count:
is_fraud
0    707413
1       923
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    177085
Name: count, dtype: Int64

Block 3
Train fraud count:
is_fraud
0    725007
1       836
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    180960
1       501
Name: count, dtype: Int64

Block 4
Train fraud count:
is_fraud
0    731394
1       664
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    183015
Name: count, dtype: Int64

Block 5
Train f

In [15]:
'''
result = pd.DataFrame(columns=[
    "Model", "Features", 
    "Train AUC", "Test AUC", 
    "Train PR AUC", "Test PR AUC"
])

'''

# ALL features

all_cols = ['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc_code',
       'errors', 'merchant_state_missing_flag', 'zip_missing_flag',
       'errors_missing_flag', 'use_chip_Chip Transaction',
       'use_chip_Online Transaction', 'use_chip_Swipe Transaction',
       'current_age', 'retirement_age', 'birth_year', 'birth_month', 'gender',
       'address', 'latitude', 'longitude', 'per_capita_income',
       'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards',
       'gender_Male', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web', 'card_type_Credit',
       'card_type_Debit', 'card_type_Debit (Prepaid)', 'card_brand_Amex',
       'card_brand_Discover', 'card_brand_Mastercard', 'card_brand_Visa',
       'has_chip_YES', 'is_fraud_missing_flag']

VIF_col = ["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction",'is_fraud_missing_flag','merchant_state_missing_flag', 'zip_missing_flag','card_on_dark_web']

identifier = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

set_VIF = set(VIF_col)
set_identifier =set(identifier)
exclude_cols = set(VIF_col) | set(identifier)
all_cols = [x for x in all_cols if x not in exclude_cols]


# RFM features
rfm_cols = [
    'RecencyInterval', 'TxnFrequency_7d','TxnFrequency_30d',
    'TxnFrequency_60d', 'TxnFrequency_90d','AmtDelta'
]

# DK features
dk_cols = [
    'merchant_online', 'merchant_us', 'merchant_eu', 'merchant_others',
    'FirstTxnInRegion','HighRiskMCC','DifferentState'
]

# Grouping
feature_groups = {
    "X_all": all_cols,
    "X_rfm": rfm_cols,
    "X_dk": dk_cols,
    "X_all + X_rfm": all_cols + rfm_cols,
    "X_all + X_dk": all_cols + dk_cols,
    "X_rfm + X_dk": rfm_cols + dk_cols,
    "X_all + X_rfm + X_dk": all_cols + rfm_cols + dk_cols
}

## 04-3 老師建議我們先省略 Assumption: Avoid Multicollinearity

In [13]:
##處理共線性
train_df.drop(columns=["per_capita_income"], inplace=True)
train_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
train_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)
#再重跑一次VIF

In [14]:
test_df.drop(columns=["per_capita_income"], inplace=True)
test_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
test_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)

## Stepwise selection

In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score




In [16]:
def stepwise_logit_score_test(
    train_df,
    test_df,
    feature_cols,
    dep_var="is_fraud",
    threshold_in=0.01,
    threshold_out=0.05,
    max_iter=30,
    verbose=False
):
    """
    Stepwise Logistic Regression using Approximate Score Test
    """

    y_train = train_df[dep_var].values
    y_test  = test_df[dep_var].values

    X_train = train_df[feature_cols].fillna(0)
    X_test  = test_df[feature_cols].fillna(0)

    # ---- Standardize ----
    scaler = StandardScaler()
    X_train_s = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=feature_cols,
        index=train_df.index
    )
    X_test_s = pd.DataFrame(
        scaler.transform(X_test),
        columns=feature_cols,
        index=test_df.index
    )

    included = []
    candidates = list(feature_cols)

    for _ in range(max_iter):

        changed = False

        # ==========================
        # Forward (Score Test)
        # ==========================
        if included:
            X_base = sm.add_constant(X_train_s[included], has_constant="add")
        else:
            X_base = sm.add_constant(
                pd.DataFrame(index=X_train_s.index),
                has_constant="add"
            )

        base_model = sm.Logit(y_train, X_base).fit(disp=False)
        p_hat = base_model.predict(X_base)
        resid = y_train - p_hat

        score_pvals = {}

        for var in candidates:
            score = np.dot(resid, X_train_s[var])
            info  = np.dot(p_hat * (1 - p_hat), X_train_s[var] ** 2)
            if info <= 0:
                continue
            z = score / np.sqrt(info)
            pval = 2 * (1 - sm.stats.norm.cdf(abs(z)))
            score_pvals[var] = pval

        if score_pvals:
            best_var = min(score_pvals, key=score_pvals.get)
            if score_pvals[best_var] < threshold_in:
                included.append(best_var)
                candidates.remove(best_var)
                changed = True
                if verbose:
                    print(f"  + Add {best_var} (p={score_pvals[best_var]:.4g})")

        # ==========================
        # Backward (Wald Test)
        # ==========================
        if included:
            X_full = sm.add_constant(X_train_s[included], has_constant="add")
            full_model = sm.Logit(y_train, X_full).fit(disp=False)

            pvals = full_model.pvalues.drop("const")
            worst_pval = pvals.max()

            if worst_pval > threshold_out:
                worst_var = pvals.idxmax()
                included.remove(worst_var)
                candidates.append(worst_var)
                changed = True
                if verbose:
                    print(f"  - Drop {worst_var} (p={worst_pval:.4g})")

        if not changed:
            break

    # ==========================
    # Final Model
    # ==========================
    if not included:
        return {"status": "error", "message": "No variable selected"}

    X_train_f = sm.add_constant(X_train_s[included], has_constant="add")
    X_test_f  = sm.add_constant(X_test_s[included], has_constant="add")

    final_model = sm.Logit(y_train, X_train_f).fit(disp=False)

    train_prob = final_model.predict(X_train_f)
    test_prob  = final_model.predict(X_test_f)

    return {
        "status": "success",
        "selected_features": included,
        "Train AUC": round(roc_auc_score(y_train, train_prob), 4),
        "Test AUC": round(roc_auc_score(y_test, test_prob), 4),
        "Train PR-AUC": round(average_precision_score(y_train, train_prob), 4),
        "Test PR-AUC": round(average_precision_score(y_test, test_prob), 4)
    }


In [17]:
full_logit_results = {}

for block_id, block_df in df_sorted.groupby("time_block"):

    print(f"\n{'='*60}")
    print(f"🚀 Block {block_id} | Year {2010 + block_id*2}-{2011 + block_id*2}")
    print(f"{'='*60}")

    block_df = block_df.sort_values("date")
    split_idx = int(len(block_df) * 0.8)

    train_raw = block_df.iloc[:split_idx].copy()
    test_raw  = block_df.iloc[split_idx:].copy()

    # ---- HighRiskMCC (ANTI-LEAKAGE) ----
    fraud_rate = train_raw.groupby("mcc_code")["is_fraud"].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index

    train_raw["HighRiskMCC"] = train_raw["mcc_code"].isin(high_risk_mcc).astype("uint8")
    test_raw["HighRiskMCC"]  = test_raw["mcc_code"].isin(high_risk_mcc).astype("uint8")

    full_logit_results[block_id] = {}

    for group_name, feature_list in feature_groups.items():

        valid_features = [f for f in feature_list if f in train_raw.columns]
        if not valid_features:
            continue

        print(f"   🔹 {group_name} ({len(valid_features)} vars)...", end=" ")

        res = stepwise_logit_score_test(
            train_raw,
            test_raw,
            valid_features,
            dep_var="is_fraud",
            verbose=False
        )

        if res["status"] == "success":
            print(f"✅ Test PR-AUC={res['Test PR-AUC']}")
            full_logit_results[block_id][group_name] = res
        else:
            print("❌ Failed")
            full_logit_results[block_id][group_name] = {"error": res["message"]}



🚀 Block 0 | Year 2010-2011
   🔹 X_all (26 vars)... 

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.UInt8DType'>, <class 'numpy.dtypes.UInt8DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.UInt8DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.

In [ ]:
data_list = []

for block_id, groups in full_logit_results.items():
    for group_name, metrics in groups.items():
        if "error" in metrics:
            continue

        data_list.append({
            "Block": block_id,
            "Year": f"{2010 + block_id*2}-{2011 + block_id*2}",
            "Feature Group": group_name,
            "Train AUC": metrics["Train AUC"],
            "Test AUC": metrics["Test AUC"],
            "Train PR-AUC": metrics["Train PR-AUC"],
            "Test PR-AUC": metrics["Test PR-AUC"]
        })

df_results = pd.DataFrame(data_list)


## LR all

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import copy

# ===========================================================
# 1. Full Logistic Regression 訓練函數
# ===========================================================
def fit_full_logit(train_df, test_df, feature_cols, dep_var="is_fraud"):
    """
    針對給定的特徵列表進行全變數 Logistic Regression (L2 Penalty)
    """
    # 準備 X 和 y
    X_train = train_df[feature_cols]
    y_train = train_df[dep_var]

    X_test = test_df[feature_cols]
    y_test = test_df[dep_var]

    # 處理缺失值 (Sklearn 不接受 NaN)
    # 簡單策略：填補 0 或刪除 (這邊假設你前面的資料清洗已經處理完 NaN，若有殘留建議 fillna)
    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)

    # 1. 標準化 (Standardization) - 非常重要！
    # 必須用 Train 的數據來 fit，然後 transform 到 Train 和 Test
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 2. 訓練模型 (使用 L2 Regularization 防止過擬合)
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=2000,  # 增加迭代次數避免收斂失敗
        n_jobs=-1,      # 平行運算
        random_state=42
    )
    
    try:
        model.fit(X_train_scaled, y_train)
        
        # 3. 預測機率
        train_pred_prob = model.predict_proba(X_train_scaled)[:, 1]
        test_pred_prob = model.predict_proba(X_test_scaled)[:, 1]

        # 4. 計算指標
        train_auc = roc_auc_score(y_train, train_pred_prob)
        test_auc = roc_auc_score(y_test, test_pred_prob)
        
        train_prauc = average_precision_score(y_train, train_pred_prob)
        test_prauc = average_precision_score(y_test, test_pred_prob)
        
        return {
            "status": "success",
            "model": model,
            "scaler": scaler,
            "metrics": {
                "Train AUC": round(train_auc, 4),
                "Test AUC": round(test_auc, 4),
                "Train PR-AUC": round(train_prauc, 4),
                "Test PR-AUC": round(test_prauc, 4)
            }
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


# ===========================================================
# 2. 準備 Feature Groups (確保無目標變數與ID)
# ===========================================================
# 確保 all_cols, rfm_cols, dk_cols 已經定義
identifier = ['transaction_id', 'client_id_x', 'card_id', 'card_number', 'cvv']
exclude_cols = set(identifier) | {'is_fraud', 'date', 'year', 'time_block'}

# 這裡假設你的 cols 列表已經存在，做最後一次清洗確保安全
# 如果你之前的 all_cols 已經清乾淨了，這裡只是雙重保險
def clean_cols(cols):
    return [c for c in cols if c not in exclude_cols]

feature_groups = {
    "X_all": clean_cols(all_cols),
    "X_rfm": clean_cols(rfm_cols),
    "X_dk": clean_cols(dk_cols),
    "X_all + X_rfm": clean_cols(all_cols + rfm_cols),
    "X_all + X_dk": clean_cols(all_cols + dk_cols),
    "X_rfm + X_dk": clean_cols(rfm_cols + dk_cols),
    "X_all + X_rfm + X_dk": clean_cols(all_cols + rfm_cols + dk_cols)
}


# ===========================================================
# 3. 主迴圈 (Block + Feature Groups)
# ===========================================================
full_logit_results = {}

for block_id, block_df in df_sorted.groupby('time_block'):
    
    print(f"\n{'='*50}")
    print(f"🚀 Running Full Logit for Block {block_id} (Year: {2010 + block_id*2})")
    print(f"{'='*50}")

    # --- 資料切分 (Split) ---
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # --- 重算 HighRiskMCC (Anti-Leakage) ---
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index
    
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')

    # 初始化結果存放
    full_logit_results[block_id] = {}

    # --- 針對每個 Feature Group 跑模型 ---
    for group_name, feature_list in feature_groups.items():
        
        # 確保特徵存在於 DataFrame
        valid_features = [f for f in feature_list if f in train_raw.columns]
        
        # 簡單檢查：若特徵少於 1 個就跳過
        if not valid_features:
            print(f"   ⚠️ Group: {group_name} - No valid features found. Skipping.")
            continue
            
        print(f"   🔹 Group: {group_name} ({len(valid_features)} features)...", end=" ")

        # 執行 Full Logistic Regression
        res = fit_full_logit(
            train_raw, 
            test_raw, 
            feature_cols=valid_features, 
            dep_var="is_fraud"
        )

        if res["status"] == "success":
            metrics = res["metrics"]
            print(f"✅ Done.")
            print(f"      Train AUC: {metrics['Train AUC']} | Test AUC: {metrics['Test AUC']}")
            print(f"      Train PR : {metrics['Train PR-AUC']} | Test PR : {metrics['Test PR-AUC']}")
            
            # 儲存結果
            full_logit_results[block_id][group_name] = metrics
        else:
            print(f"❌ Error: {res['message']}")
            full_logit_results[block_id][group_name] = {"error": res['message']}

print("\n🎉 All Full Logistic Regression models completed!")


🚀 Running Full Logit for Block 0 (Year: 2010)
   🔹 Group: X_all (25 features)... ✅ Done.
      Train AUC: 0.9534 | Test AUC: 0.9517
      Train PR : 0.0572 | Test PR : 0.0524
   🔹 Group: X_rfm (6 features)... ✅ Done.
      Train AUC: 0.6884 | Test AUC: 0.7442
      Train PR : 0.0098 | Test PR : 0.026
   🔹 Group: X_dk (7 features)... ✅ Done.
      Train AUC: 0.9716 | Test AUC: 0.9736
      Train PR : 0.1047 | Test PR : 0.106
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done.
      Train AUC: 0.9599 | Test AUC: 0.9617
      Train PR : 0.0851 | Test PR : 0.1219
   🔹 Group: X_all + X_dk (32 features)... ✅ Done.
      Train AUC: 0.9767 | Test AUC: 0.9773
      Train PR : 0.16 | Test PR : 0.1842
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done.
      Train AUC: 0.9778 | Test AUC: 0.9807
      Train PR : 0.1357 | Test PR : 0.2014
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done.
      Train AUC: 0.9798 | Test AUC: 0.9815
      Train PR : 0.2039 | Test PR : 0.2456

🚀 Running Full Logi

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.9675 | Test AUC: nan
      Train PR : 0.0025 | Test PR : 0.0
   🔹 Group: X_rfm (6 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.7059 | Test AUC: nan
      Train PR : 0.0005 | Test PR : 0.0
   🔹 Group: X_dk (7 features)... 

...

✅ Done.
      Train AUC: 0.9068 | Test AUC: nan
      Train PR : 0.0871 | Test PR : 0.0
   🔹 Group: X_rfm + X_dk (13 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.9214 | Test AUC: nan
      Train PR : 0.1213 | Test PR : 0.0
   🔹 Group: X_all + X_rfm + X_dk (38 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done.
      Train AUC: 0.9152 | Test AUC: nan
      Train PR : 0.1209 | Test PR : 0.0

🚀 Running Full Logit for Block 5 (Year: 2020)
   🔹 Group: X_all (25 features)... ✅ Done.
      Train AUC: 0.9198 | Test AUC: 0.9051
      Train PR : 0.0664 | Test PR : 0.089
   🔹 Group: X_rfm (6 features)... ✅ Done.
      Train AUC: 0.7469 | Test AUC: 0.745
      Train PR : 0.01 | Test PR : 0.0176
   🔹 Group: X_dk (7 features)... ✅ Done.
      Train AUC: 0.8684 | Test AUC: 0.8624
      Train PR : 0.1043 | Test PR : 0.1276
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done.
      Train AUC: 0.9377 | Test AUC: 0.9248
      Train PR : 0.1265 | Test PR : 0.1728
   🔹 Group: X_all + X_dk (32 features)... ✅ Done.
      Train AUC: 0.9135 | Test AUC: 0.9005
      Train PR : 0.1516 | Test PR : 0.1755
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done.
      Train AUC: 0.9179 | Test AUC: 0.9029
      Train PR : 0.1797 | Test PR : 0.2564
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done.
      Train AUC: 0

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ==========================================
# 1. 將巢狀字典轉為 DataFrame
# ==========================================
data_list = []

for block_id, groups in full_logit_results.items():
    for group_name, metrics in groups.items():
        # 排除發生錯誤的組別
        if "error" in metrics:
            continue
            
        row = {
            "Block": block_id,
            "Year": f"{2010 + block_id}",
            "Feature Group": group_name,
            "Train AUC": metrics.get("Train AUC"),
            "Test AUC": metrics.get("Test AUC"),
            "Train PR-AUC": metrics.get("Train PR-AUC"),
            "Test PR-AUC": metrics.get("Test PR-AUC")
        }
        data_list.append(row)

df_results = pd.DataFrame(data_list)

# 調整欄位順序
cols = ["Block", "Year", "Feature Group", "Train AUC", "Test AUC", "Train PR-AUC", "Test PR-AUC"]
df_results = df_results[cols]

# 顯示前幾筆
#print("=== 詳細結果總表 (前 5 筆) ===")
#print(df_results.head())

# ==========================================
# 2. 製作 Test AUC 比較矩陣 (Pivot Table)
# ==========================================
# 這是最直觀的表格：橫軸是時間，縱軸是特徵群組，數值是 Test AUC
train_auc = df_results.pivot(index="Feature Group", columns="Year", values="Train AUC")
test_auc = df_results.pivot(index="Feature Group", columns="Year", values="Test AUC")
train_prauc = df_results.pivot(index="Feature Group", columns="Year", values="Train PR-AUC")
test_prauc = df_results.pivot(index="Feature Group", columns="Year", values="Test PR-AUC")


print("\n=== Traing AUC 績效矩陣 (數值越高越好) ===")
print(train_auc)
print("\n=== Test AUC 績效矩陣 (數值越高越好) ===")
print(test_auc)
print("\n=== Traing PR-AUC 績效矩陣 (數值越高越好) ===")
print(train_prauc)
print("\n=== Test PR-AUC 績效矩陣 (數值越高越好) ===")
print(test_prauc)


=== Traing AUC 績效矩陣 (數值越高越好) ===
Year                    2010    2011    2012    2013    2014    2015    2016  \
Feature Group                                                                  
X_all                 0.9534  0.9675  0.8455  0.9233  0.9153  0.9198  0.9169   
X_all + X_dk          0.9767  0.9797  0.8389  0.9118  0.9068  0.9135  0.9171   
X_all + X_rfm         0.9599  0.9696  0.8888  0.9417  0.9281  0.9377  0.9329   
X_all + X_rfm + X_dk  0.9798  0.9843  0.8904  0.9442  0.9152  0.9370  0.9313   
X_dk                  0.9716  0.9631  0.8086  0.8760  0.8708  0.8684  0.8722   
X_rfm                 0.6884  0.7059  0.7156  0.7735  0.7351  0.7469  0.7122   
X_rfm + X_dk          0.9778  0.9743  0.8904  0.9315  0.9214  0.9179  0.9150   

Year                    2018    2019  
Feature Group                         
X_all                 0.9799  0.9780  
X_all + X_dk          0.9992  0.9991  
X_all + X_rfm         0.9821  0.9806  
X_all + X_rfm + X_dk  0.9994  0.9992  
X_dk       

In [21]:
# ==========================================
# 2. 製作整合大表 (Hierarchical Columns)
# ==========================================

# 1. 設定要轉置的所有指標
metrics = ["Train AUC", "Test AUC", "Train PR-AUC", "Test PR-AUC"]

# 2. 使用 pivot 一次處理多個 values
# 這會產生一個 MultiIndex，第一層是指標 (Metric)，第二層是年份 (Year)
df_pivot = df_results.pivot(index="Feature Group", columns="Year", values=metrics)

# 3. 交換欄位層級：將「年份」移到最上層，指標移到第二層
df_pivot.columns = df_pivot.columns.swaplevel(0, 1)

# 4. (選擇性) 排序欄位
# 如果直接 sort_index，欄位會依照字母排序 (Test 會跑去 Train 前面)
# 為了跟你的截圖一樣 (Train -> Test)，我們需要自定義排序
unique_years = sorted(df_results["Year"].unique())
ordered_columns = []

for year in unique_years:
    for metric in metrics:
        # 建立 (Year, Metric) 的 tuple 列表
        ordered_columns.append((year, metric))

# 根據我們想要的順序重新索引
df_final = df_pivot.reindex(columns=ordered_columns)

print("=== 整合結果總表 ===")
# 如果你在 Jupyter Notebook，直接輸入 df_final 即可看到漂亮表格
# 如果是純文字介面，用 to_string() 比較清楚
print(df_final.to_string())

# 簡單的匯出 csv 檢查 (通常 Excel 打開就會有雙層標頭)
df_final.to_csv("logit_results_wide.csv")

=== 整合結果總表 ===
Year                      2010                                        2011                                        2012                                        2013                                        2014                                        2015                                        2016                                        2018                                        2019                                  
                     Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC
Feature Group                                                                                                                                          

## XGB

In [22]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
import copy

# ===========================================================
# 1. 定義單次 XGBoost 訓練函數
# ===========================================================
def run_xgb_single(train_df, test_df, feature_cols, dep_var="is_fraud"):
    """
    針對給定的 Train/Test 和特徵列表訓練 XGBoost
    """
    X_train = train_df[feature_cols]
    y_train = train_df[dep_var]
    X_test = test_df[feature_cols]
    y_test = test_df[dep_var]

    # 初始化 XGBoost
    model = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="auc", # 改用 AUC 作為評估指標通常比較直觀，logloss 也可以
        random_state=42,
        tree_method="hist", # 加速訓練
        n_jobs=-1,
        early_stopping_rounds=50 # 如果 50 輪內驗證集 AUC 沒提升就停止
    )

    try:
        # 訓練模型
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            verbose=False # 關閉詳細訓練過程輸出，避免洗版
        )

        # 預測機率
        train_pred = model.predict_proba(X_train)[:, 1]
        test_pred = model.predict_proba(X_test)[:, 1]

        # 計算指標
        train_roc = roc_auc_score(y_train, train_pred)
        test_roc = roc_auc_score(y_test, test_pred)
        train_pr = average_precision_score(y_train, train_pred)
        test_pr = average_precision_score(y_test, test_pred)

        # 取得最佳迭代次數 (如果觸發早停)
        best_iter = model.best_iteration if hasattr(model, 'best_iteration') else 300

        return {
            "status": "success",
            "model": model,
            "metrics": {
                "Train AUC": round(train_roc, 4),
                "Test AUC": round(test_roc, 4),
                "Train PR-AUC": round(train_pr, 4),
                "Test PR-AUC": round(test_pr, 4),
                "Best Iteration": best_iter
            }
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}


# ===========================================================
# 2. 準備 Feature Groups (確保無目標變數與ID)
# ===========================================================
# 假設 identifier, all_cols, rfm_cols, dk_cols 已經定義好
identifier = ['transaction_id', 'client_id_x', 'card_id', 'card_number', 'cvv']
exclude_cols = set(identifier) | {'is_fraud', 'date', 'year', 'time_block'}

def clean_cols(cols):
    return [c for c in cols if c not in exclude_cols]

feature_groups = {
    "X_all": clean_cols(all_cols),
    "X_rfm": clean_cols(rfm_cols),
    "X_dk": clean_cols(dk_cols),
    "X_all + X_rfm": clean_cols(all_cols + rfm_cols),
    "X_all + X_dk": clean_cols(all_cols + dk_cols),
    "X_rfm + X_dk": clean_cols(rfm_cols + dk_cols),
    "X_all + X_rfm + X_dk": clean_cols(all_cols + rfm_cols + dk_cols)
}

# ===========================================================
# 3. 主迴圈 (Block + Feature Groups)
# ===========================================================
xgb_results = {}

# 確保 df_sorted 存在
if 'df_sorted' not in locals():
    raise ValueError("df_sorted is not defined. Please ensure your DataFrame is prepared.")

for block_id, block_df in df_sorted.groupby('time_block'):
    
    print(f"\n{'='*60}")
    print(f"🚀 Running XGBoost for Block {block_id} (Year: {2010 + block_id})")
    print(f"{'='*60}")

    # --- 資料切分 (Split) ---
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # --- 重算 HighRiskMCC (Anti-Leakage) ---
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index
    
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')

    # 初始化結果存放
    xgb_results[block_id] = {}

    # --- 針對每個 Feature Group 跑模型 ---
    for group_name, feature_list in feature_groups.items():
        
        # 確保特徵存在於 DataFrame
        valid_features = [f for f in feature_list if f in train_raw.columns]
        
        # 簡單檢查
        if not valid_features:
            print(f"   ⚠️ Group: {group_name} - No valid features found. Skipping.")
            continue
            
        print(f"   🔹 Group: {group_name} ({len(valid_features)} features)...", end=" ")

        # 執行 XGBoost
        res = run_xgb_single(
            train_raw, 
            test_raw, 
            feature_cols=valid_features, 
            dep_var="is_fraud"
        )

        if res["status"] == "success":
            metrics = res["metrics"]
            print(f"✅ Done (Best Iter: {metrics['Best Iteration']})")
            print(f"      Train AUC: {metrics['Train AUC']} | Test AUC: {metrics['Test AUC']}")
            print(f"      Train PR : {metrics['Train PR-AUC']} | Test PR : {metrics['Test PR-AUC']}")
            
            xgb_results[block_id][group_name] = metrics
        else:
            print(f"❌ Error: {res['message']}")
            xgb_results[block_id][group_name] = {"error": res['message']}

print("\n🎉 All XGBoost models completed!")


🚀 Running XGBoost for Block 0 (Year: 2010)
   🔹 Group: X_all (25 features)... ✅ Done (Best Iter: 216)
      Train AUC: 0.9999 | Test AUC: 0.9988
      Train PR : 0.9812 | Test PR : 0.9059
   🔹 Group: X_rfm (6 features)... ✅ Done (Best Iter: 114)
      Train AUC: 0.8944 | Test AUC: 0.8317
      Train PR : 0.0907 | Test PR : 0.035
   🔹 Group: X_dk (7 features)... ✅ Done (Best Iter: 33)
      Train AUC: 0.9749 | Test AUC: 0.9766
      Train PR : 0.1461 | Test PR : 0.1537
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done (Best Iter: 212)
      Train AUC: 0.9999 | Test AUC: 0.9992
      Train PR : 0.9866 | Test PR : 0.9253
   🔹 Group: X_all + X_dk (32 features)... ✅ Done (Best Iter: 159)
      Train AUC: 0.9998 | Test AUC: 0.9992
      Train PR : 0.9767 | Test PR : 0.924
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done (Best Iter: 108)
      Train AUC: 0.9925 | Test AUC: 0.9907
      Train PR : 0.5279 | Test PR : 0.4573
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done (Best Iter: 2

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [12:11:12] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [12:11:13] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.c

✅ Done (Best Iter: 0)
      Train AUC: 0.9651 | Test AUC: nan
      Train PR : 0.0008 | Test PR : 0.0
   🔹 Group: X_rfm (6 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [12:11:14] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 0)
      Train AUC: 0.8901 | Test AUC: nan
      Train PR : 0.0003 | Test PR : 0.0
   🔹 Group: X_dk (7 features)... 

...

✅ Done (Best Iter: 0)
      Train AUC: 0.9831 | Test AUC: nan
      Train PR : 0.0666 | Test PR : 0.0
   🔹 Group: X_rfm + X_dk (13 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [12:11:43] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 0)
      Train AUC: 0.9323 | Test AUC: nan
      Train PR : 0.0228 | Test PR : 0.0
   🔹 Group: X_all + X_rfm + X_dk (38 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [12:11:43] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/xgboost/callback.py:266: UserWarning: [12:11:44] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.c

✅ Done (Best Iter: 0)
      Train AUC: 0.9813 | Test AUC: nan
      Train PR : 0.0559 | Test PR : 0.0

🚀 Running XGBoost for Block 5 (Year: 2015)
   🔹 Group: X_all (25 features)... ✅ Done (Best Iter: 289)
      Train AUC: 0.9993 | Test AUC: 0.9656
      Train PR : 0.9433 | Test PR : 0.754
   🔹 Group: X_rfm (6 features)... ✅ Done (Best Iter: 74)
      Train AUC: 0.8997 | Test AUC: 0.8035
      Train PR : 0.0684 | Test PR : 0.0224
   🔹 Group: X_dk (7 features)... ✅ Done (Best Iter: 126)
      Train AUC: 0.9217 | Test AUC: 0.9122
      Train PR : 0.1048 | Test PR : 0.1287
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done (Best Iter: 296)
      Train AUC: 0.9997 | Test AUC: 0.9779
      Train PR : 0.9755 | Test PR : 0.7672
   🔹 Group: X_all + X_dk (32 features)... ✅ Done (Best Iter: 209)
      Train AUC: 0.9985 | Test AUC: 0.9759
      Train PR : 0.9226 | Test PR : 0.7692
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done (Best Iter: 123)
      Train AUC: 0.9823 | Test AUC: 0.9496
      Trai

In [24]:
# 整理 XGBoost 結果表格
xgb_data_list = []

for block_id, groups in xgb_results.items():
    for group_name, metrics in groups.items():
        if "error" in metrics: continue
        row = {
            "Block": block_id,
            "Year": f"{2010 + block_id*2}-{2011 + block_id*2}",
            "Feature Group": group_name,
            "Train AUC": metrics.get("Train AUC"),
            "Test AUC": metrics.get("Test AUC"),
            "Train PR-AUC": metrics.get("Train PR-AUC"),
            "Test PR-AUC": metrics.get("Test PR-AUC"),
            "Best Iter": metrics.get("Best Iteration")
        }
        xgb_data_list.append(row)

df_xgb = pd.DataFrame(xgb_data_list)
print(df_xgb.head())

# 製作 Test AUC 矩陣
pivot_trxgb = df_xgb.pivot(index="Feature Group", columns="Year", values="Train PR-AUC")
pivot_testxgb = df_xgb.pivot(index="Feature Group", columns="Year", values="Test PR-AUC")
print("\n=== XGBoost Train PR-AUC Matrix ===")
print(pivot_trxgb)
print("\n")
print("\n=== XGBoost Test PR-AUC Matrix ===")
print(pivot_testxgb)

   Block       Year  Feature Group  Train AUC  Test AUC  Train PR-AUC  \
0      0  2010-2011          X_all     0.9878       NaN        0.2217   
1      0  2010-2011          X_rfm     0.7702       NaN        0.0084   
2      0  2010-2011           X_dk     0.9743       NaN        0.1308   
3      0  2010-2011  X_all + X_rfm     0.9793       NaN        0.0874   
4      0  2010-2011   X_all + X_dk     0.9946       NaN        0.2525   

   Test PR-AUC  Best Iter  
0          0.0          0  
1          0.0          0  
2          0.0          0  
3          0.0          0  
4          0.0          0  

=== XGBoost Train PR-AUC Matrix ===
Year                  2010-2011  2012-2013  2014-2015  2016-2017  2018-2019
Feature Group                                                              
X_all                    0.2217     0.6527     0.9058     0.3223     0.9393
X_all + X_dk             0.2525     0.6197     0.8607     0.5034     0.9234
X_all + X_rfm            0.0874     0.7646     0.968

In [25]:
# ==========================================
# 4. 將 XGBoost 結果轉換為 DataFrame
# ==========================================
xgb_data_list = []

for block_id, groups in xgb_results.items():
    # 根據你的 code 邏輯：Year = 2010 + block_id
    current_year = 2010 + block_id 
    
    for group_name, metrics in groups.items():
        # 排除發生錯誤的組別
        if "error" in metrics:
            continue
            
        row = {
            "Year": current_year,
            "Feature Group": group_name,
            "Train AUC": metrics.get("Train AUC"),
            "Test AUC": metrics.get("Test AUC"),
            "Train PR-AUC": metrics.get("Train PR-AUC"),
            "Test PR-AUC": metrics.get("Test PR-AUC"),
            "Best Iteration": metrics.get("Best Iteration") # XGBoost 特有的資訊
        }
        xgb_data_list.append(row)

df_xgb_raw = pd.DataFrame(xgb_data_list)

# ==========================================
# 5. 製作整合大表 (Hierarchical Columns)
# ==========================================

# 設定要呈現的指標 (可以自由增減，例如加入 Best Iteration)
target_metrics = ["Train AUC", "Test AUC", "Train PR-AUC", "Test PR-AUC"]

# 1. 使用 pivot 轉置
# index: 列 (Feature Group)
# columns: 欄 (Year)
# values: 值 (所有指標)
df_xgb_pivot = df_xgb_raw.pivot(index="Feature Group", columns="Year", values=target_metrics)

# 2. 交換欄位層級：將「年份」移到最上層 (Level 0)，指標移到第二層 (Level 1)
df_xgb_pivot.columns = df_xgb_pivot.columns.swaplevel(0, 1)

# 3. 重新排序欄位
# 我們希望年份從小到大，且每個年份內的指標順序固定 (Train -> Test)
unique_years = sorted(df_xgb_raw["Year"].unique())
ordered_columns = []

for year in unique_years:
    for metric in target_metrics:
        ordered_columns.append((year, metric))

# 強制套用排序
df_xgb_final = df_xgb_pivot.reindex(columns=ordered_columns)

print("\n=== XGBoost 整合結果總表 ===")
# 如果你在 Jupyter/Colab，直接輸入 df_xgb_final 即可
# 這裡用 to_string 確保純文字介面也能看清楚
print(df_xgb_final.to_string())

# ==========================================
# 6. (選用) 視覺化樣式 - 針對 Jupyter Notebook
# ==========================================
def highlight_xgb(val):
    if pd.isna(val): return ''
    # 根據經驗，XGBoost 的 Train AUC 很容易接近 1，所以標準可以設高一點
    if val > 0.95: return 'background-color: #d4edda; color: green; font-weight: bold' # 極好 (綠)
    if val < 0.70: return 'background-color: #f8d7da; color: red'   # 表現差 (紅)
    return ''

# 顯示帶有顏色的表格
# df_xgb_final.style.applymap(highlight_xgb).format("{:.4f}")
df_xgb_final.to_csv("XGB_results_wide.csv")


=== XGBoost 整合結果總表 ===
Year                      2010                                        2011                                        2012                                        2013                                        2014                                        2015                                        2016                                        2018                                        2019                                  
                     Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC
Feature Group                                                                                                                                 

## Light GBM

In [26]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score

# ===========================================================
# 1. 定義單次 LightGBM 訓練函數 (標準化介面)
# ===========================================================
def run_lgbm_single(train_df, test_df, feature_cols, dep_var="is_fraud"):
    """
    針對給定的 Train/Test 和特徵列表訓練 LightGBM
    """
    X_train = train_df[feature_cols]
    y_train = train_df[dep_var]
    X_test = test_df[feature_cols]
    y_test = test_df[dep_var]

    # --- 處理類別不平衡 (Class Imbalance) ---
    pos = y_train.sum()
    neg = len(y_train) - pos
    spw = neg / pos if pos > 0 else 1

    # --- 初始化模型 (保留您原本的參數設定) ---
    model = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.02,
        max_bin=128,
        scale_pos_weight=spw,
        min_split_gain=1.0,
        reg_alpha=0.1,
        reg_lambda=5,
        min_child_samples=20,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1 # 減少干擾輸出
    )

    try:
        # --- 訓練模型 ---
        # 加入 early_stopping 以防止過擬合 (需搭配 eval_set)
        callbacks = [lgb.early_stopping(stopping_rounds=50, verbose=False)]
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="auc",
            callbacks=callbacks
        )

        # --- 預測機率 ---
        train_pred = model.predict_proba(X_train)[:, 1]
        test_pred = model.predict_proba(X_test)[:, 1]

        # --- 計算指標 ---
        train_roc = roc_auc_score(y_train, train_pred)
        test_roc = roc_auc_score(y_test, test_pred)
        train_pr = average_precision_score(y_train, train_pred)
        test_pr = average_precision_score(y_test, test_pred)
        
        # 取得最佳迭代次數
        best_iter = model.best_iteration_ if model.best_iteration_ else 500

        return {
            "status": "success",
            "model": model,
            "metrics": {
                "Train AUC": round(train_roc, 4),
                "Test AUC": round(test_roc, 4),
                "Train PR-AUC": round(train_pr, 4),
                "Test PR-AUC": round(test_pr, 4),
                "Best Iteration": best_iter
            }
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

# ===========================================================
# 2. 主迴圈 (Block + Feature Groups)
# ===========================================================
lgbm_results = {}

# 確保 df_sorted 存在
if 'df_sorted' not in locals():
    raise ValueError("df_sorted is not defined. Please ensure your DataFrame is prepared.")

# 定義特徵群組 (若上面已定義可省略，這裡為了完整性再次列出)
# 假設 identifier, all_cols, rfm_cols, dk_cols 已經定義好
identifier = ['transaction_id', 'client_id_x', 'card_id', 'card_number', 'cvv']
exclude_cols = set(identifier) | {'is_fraud', 'date', 'year', 'time_block'}

def clean_cols(cols):
    return [c for c in cols if c not in exclude_cols]

# 請確保這些變數 (all_cols 等) 在您的環境中已定義
feature_groups = {
    "X_all": clean_cols(all_cols),
    "X_rfm": clean_cols(rfm_cols),
    "X_dk": clean_cols(dk_cols),
    "X_all + X_rfm": clean_cols(all_cols + rfm_cols),
    "X_all + X_dk": clean_cols(all_cols + dk_cols),
    "X_rfm + X_dk": clean_cols(rfm_cols + dk_cols),
    "X_all + X_rfm + X_dk": clean_cols(all_cols + rfm_cols + dk_cols)
}

for block_id, block_df in df_sorted.groupby('time_block'):
    
    print(f"\n{'='*60}")
    print(f"🚀 Running LightGBM for Block {block_id} (Year: {2010 + block_id})")
    print(f"{'='*60}")

    # --- 資料切分 (Split) ---
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # --- 重算 HighRiskMCC (Anti-Leakage) ---
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index
    
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')

    # 初始化該區塊結果
    lgbm_results[block_id] = {}

    # --- 針對每個 Feature Group 跑模型 ---
    for group_name, feature_list in feature_groups.items():
        
        # 簡單檢查特徵是否存在
        valid_features = [f for f in feature_list if f in train_raw.columns]
        if not valid_features:
            continue
            
        print(f"   🔹 Group: {group_name} ({len(valid_features)} features)...", end=" ")

        # 執行 LightGBM
        res = run_lgbm_single(
            train_raw, 
            test_raw, 
            feature_cols=valid_features, 
            dep_var="is_fraud"
        )

        if res["status"] == "success":
            metrics = res["metrics"]
            print(f"✅ Done (Best Iter: {metrics['Best Iteration']})")
            # 簡化顯示
            # print(f"      Train AUC: {metrics['Train AUC']} | Test AUC: {metrics['Test AUC']}")
            
            lgbm_results[block_id][group_name] = metrics
        else:
            print(f"❌ Error: {res['message']}")
            lgbm_results[block_id][group_name] = {"error": res['message']}

print("\n🎉 All LightGBM models completed!")


🚀 Running LightGBM for Block 0 (Year: 2010)
   🔹 Group: X_all (25 features)... ✅ Done (Best Iter: 74)
   🔹 Group: X_rfm (6 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_dk (7 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_dk (32 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done (Best Iter: 1)

🚀 Running LightGBM for Block 1 (Year: 2011)
   🔹 Group: X_all (25 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 1)
   🔹 Group: X_rfm (6 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 1)
   🔹 Group: X_dk (7 features)... 

...

✅ Done (Best Iter: 1)
   🔹 Group: X_rfm + X_dk (13 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_rfm + X_dk (38 features)... 

/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:1046: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


✅ Done (Best Iter: 1)

🚀 Running LightGBM for Block 5 (Year: 2015)
   🔹 Group: X_all (25 features)... ✅ Done (Best Iter: 405)
   🔹 Group: X_rfm (6 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_dk (7 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done (Best Iter: 482)
   🔹 Group: X_all + X_dk (32 features)... ✅ Done (Best Iter: 282)
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done (Best Iter: 235)
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done (Best Iter: 490)

🚀 Running LightGBM for Block 6 (Year: 2016)
   🔹 Group: X_all (25 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_rfm (6 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_dk (7 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_rfm (31 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_dk (32 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_rfm + X_dk (13 features)... ✅ Done (Best Iter: 1)
   🔹 Group: X_all + X_rfm + X_dk (38 features)... ✅ Done (Best Iter: 1)

🚀 Running L

In [28]:
# ==========================================
# 3. 製作 LightGBM 整合報表
# ==========================================
lgbm_data_list = []

for block_id, groups in lgbm_results.items():
    current_year = 2010 + block_id 
    
    for group_name, metrics in groups.items():
        if "error" in metrics:
            continue
            
        row = {
            "Year": current_year,
            "Feature Group": group_name,
            "Train AUC": metrics.get("Train AUC"),
            "Test AUC": metrics.get("Test AUC"),
            "Train PR-AUC": metrics.get("Train PR-AUC"),
            "Test PR-AUC": metrics.get("Test PR-AUC")
        }
        lgbm_data_list.append(row)

df_lgbm_raw = pd.DataFrame(lgbm_data_list)

# --- 轉置與階層化 ---
target_metrics = ["Train AUC", "Test AUC", "Train PR-AUC", "Test PR-AUC"]

# 1. Pivot: Index=Feature Group, Cols=Year
df_lgbm_pivot = df_lgbm_raw.pivot(index="Feature Group", columns="Year", values=target_metrics)

# 2. Swaplevel: 讓年份在最上層
df_lgbm_pivot.columns = df_lgbm_pivot.columns.swaplevel(0, 1)

# 3. Reindex: 強制排序 (年份小->大, 指標 Train->Test)
unique_years = sorted(df_lgbm_raw["Year"].unique())
ordered_columns = []
for year in unique_years:
    for metric in target_metrics:
        ordered_columns.append((year, metric))

df_lgbm_final = df_lgbm_pivot.reindex(columns=ordered_columns)

print("\n=== LightGBM 整合結果總表 ===")
print(df_lgbm_final.to_string())

# 如果您在 Jupyter Notebook，可以用這行顯示漂亮的 HTML 表格
# df_lgbm_final.style.format("{:.4f}").background_gradient(cmap='Blues', subset=pd.IndexSlice[:, (slice(None), 'Test AUC')])
df_lgbm_final.to_csv("lgbm.csv")


=== LightGBM 整合結果總表 ===
Year                      2010                                        2011                                        2012                                        2013                                        2014                                        2015                                        2016                                        2018                                        2019                                  
                     Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC Train AUC Test AUC Train PR-AUC Test PR-AUC
Feature Group                                                                                                                                